# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vac_data = pd.read_csv("weatherpy.csv")
vac_data.head()

,Unnamed: 0,name,Lat,Lng,temp,temp_max,humidity,clouds,speed
0,0,Tucson,32.2217,-110.9265,91.18,94.23,44,0,11.50
1,1,Saint-Philippe,-21.3585,55.7679,66.54,67.59,75,35,11.63
2,2,Sistranda,63.7256,8.8340,62.56,62.56,77,79,12.30
3,3,Glenwood Springs,39.5505,-107.3248,76.68,89.89,31,0,0.00
4,4,Vostok,46.4856,135.8833,64.22,64.22,54,23,1.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = vac_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = vac_data["humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_conditions = vac_data.loc[(vac_data["temp_max"]>70) & \
                                 (vac_data["temp_max"]<80) & \
                                 (vac_data["speed"]<10) &\
                                 (vac_data["clouds"]==0)]
weather_conditions.head()

,Unnamed: 0,name,Lat,Lng,temp,temp_max,humidity,clouds,speed
20,20,Jamestown,42.0970,-79.2353,72.50,73.04,72,0,4.61
27,27,Granada,37.1882,-3.6067,74.23,75.90,48,0,0.00
147,147,Beyneu,45.3167,55.2000,72.01,72.01,24,0,6.17
148,148,Rome,43.2128,-75.4557,69.89,72.73,79,0,3.44
151,151,Horsham,51.0640,-0.3272,65.19,70.02,87,0,2.30


In [6]:
# Remove rows missing data
Weather_conditions_cleaned = weather_conditions.dropna(how='all')

# Visualize
Weather_conditions_cleaned.head()

,Unnamed: 0,name,Lat,Lng,temp,temp_max,humidity,clouds,speed
20,20,Jamestown,42.0970,-79.2353,72.50,73.04,72,0,4.61
27,27,Granada,37.1882,-3.6067,74.23,75.90,48,0,0.00
147,147,Beyneu,45.3167,55.2000,72.01,72.01,24,0,6.17
148,148,Rome,43.2128,-75.4557,69.89,72.73,79,0,3.44
151,151,Horsham,51.0640,-0.3272,65.19,70.02,87,0,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = Weather_conditions_cleaned
hotel_df["Hotel Name"] = ""
hotel_df["City"] = ""
hotel_df["Country"] = ""
hotel_df.head(10)

,Unnamed: 0,name,Lat,Lng,temp,temp_max,humidity,clouds,speed,Hotel Name,City,Country
20,20,Jamestown,42.0970,-79.2353,72.50,73.04,72,0,4.61,,,
27,27,Granada,37.1882,-3.6067,74.23,75.90,48,0,0.00,,,
147,147,Beyneu,45.3167,55.2000,72.01,72.01,24,0,6.17,,,
148,148,Rome,43.2128,-75.4557,69.89,72.73,79,0,3.44,,,
151,151,Horsham,51.0640,-0.3272,65.19,70.02,87,0,2.30,,,
163,163,Nador,35.1740,-2.9287,77.25,77.25,83,0,2.30,,,
187,187,Atkinson,42.8384,-71.1470,73.08,77.68,79,0,3.44,,,
366,366,San Miguel de Tucumán,-26.8241,-65.2226,71.60,71.60,33,0,5.75,,,
406,406,Kiryat Motzkin,32.8364,35.0753,75.02,75.02,87,0,7.99,,,
521,521,Marsá Maţrūḩ,31.3525,27.2453,77.83,77.83,67,0,7.52,,,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
        "radius": 5000,
        "type": "lodging",
        "key": g_key
}

In [9]:
for index, row in weather_conditions.iterrows():

    #locations
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # add keyword to params dict
    params['location'] = f"{Lat},{Lng}"

    # assemble url and make API request
    hotels = requests.get(base_url, params=params).json()
    
    try:     
        Weather_conditions_cleaned.loc[index, 'Hotel Name'] = hotels["results"][0]['name']
        Weather_conditions_cleaned.loc[index, 'City'] = hotels["results"][0]['city']
        Weather_conditions_cleaned.loc[index, 'Country'] = hotels["results"][0]['country']
        print("hotel found")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
humidity = hotel_df["humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("Hotels.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>